## Практическое задание

<ol>
    <li>обучить рукописную сверточную нейронную сеть (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете cifar-10
    </li>
    <li>
        оценить рост точности при увеличении ширины сети (больше фильтров)
    </li>
    <li>
        оценить рост точности при увеличении глубины сети (больше слоев)
    </li>
    <li>
        *сравнить с точностью полносвязной сети для этой выборки
    </li>
</ol>

In [18]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os
import keras

In [19]:
classes = ['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

In [20]:
# установка параметров нейросети
batch_size = 32
num_classes = 10
epochs = 1
data_augmentation = True
num_predictions = 20
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'тренировочные примеры')
print(X_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


X_train = X_train / 255.0
X_test = X_test / 255.0

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


X_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [21]:
results = []

In [24]:
def train_models_depth(depth=2, epochs=2):
    # конфигурирование слоев нейросети
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
        
    for _ in range(depth):
        model.add(Conv2D(32,  (3, 3), padding='same'))
        model.add(Activation('relu'))
        
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))


    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    
    model.add(Activation('softmax'))
    
    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          shuffle=True)
    [loss, accuracy] = model.evaluate(X_test, y_test)
    print("Test loss {:.4f}, accuracy {:.2f}%".format(loss, accuracy * 100))
    
    # накопление результатов
    results.append([depth, loss, accuracy])
    
    return results

In [25]:
print(train_models_depth())

Epoch 1/2
1563/1563 [==============================] - 107s 68ms/step - loss: 1.8409 - accuracy: 0.3215 - val_loss: 1.5486 - val_accuracy: 0.4290
Epoch 2/2
313/313 [==============================] - 4s 14ms/step - loss: 1.4099 - accuracy: 0.4887
Test loss 1.4099, accuracy 48.87%
[[2, 1.4099366664886475, 0.4887000024318695]]


In [26]:
print(train_models_depth(3))

Epoch 1/2
1563/1563 [==============================] - 128s 81ms/step - loss: 1.8432 - accuracy: 0.3231 - val_loss: 1.5482 - val_accuracy: 0.4398
Epoch 2/2
313/313 [==============================] - 4s 14ms/step - loss: 1.4035 - accuracy: 0.4939
Test loss 1.4035, accuracy 49.39%
[[2, 1.4099366664886475, 0.4887000024318695], [3, 1.4035063982009888, 0.49390000104904175]]


In [28]:
print(train_models_depth(5))

Epoch 1/2
1563/1563 [==============================] - 159s 101ms/step - loss: 1.8829 - accuracy: 0.3080 - val_loss: 1.7193 - val_accuracy: 0.3749
Epoch 2/2
313/313 [==============================] - 5s 16ms/step - loss: 1.3920 - accuracy: 0.4849
Test loss 1.3920, accuracy 48.49%
[[2, 1.4099366664886475, 0.4887000024318695], [3, 1.4035063982009888, 0.49390000104904175], [10, 1.5331987142562866, 0.43779999017715454], [5, 1.3920201063156128, 0.48489999771118164]]


In [ ]:
print(train_models_depth(7))

Epoch 1/2
1563/1563 [==============================] - 167s 106ms/step - loss: 1.8615 - accuracy: 0.3081 - val_loss: 1.6430 - val_accuracy: 0.3927
Epoch 2/2
1237/1563 [======================>.......] - ETA: 32s - loss: 1.5707 - accuracy: 0.4255

In [27]:
print(train_models_depth(10))

Epoch 1/2
1563/1563 [==============================] - 215s 137ms/step - loss: 1.9095 - accuracy: 0.2820 - val_loss: 1.7972 - val_accuracy: 0.3484
Epoch 2/2
313/313 [==============================] - 6s 19ms/step - loss: 1.5332 - accuracy: 0.4378
Test loss 1.5332, accuracy 43.78%
[[2, 1.4099366664886475, 0.4887000024318695], [3, 1.4035063982009888, 0.49390000104904175], [10, 1.5331987142562866, 0.43779999017715454]]
